# Riadiace štruktúry

------------

## Inicializácia

----------

> Pre správne zobrazenie interaktívnych grafov a symbolického výpočtu v tomto notebooku je nutné mať nainštalované balíčky `matplotlib`, `numpy`, `ipympl`, `ipywidgets` a `sympy` (Inštalácia pomocou *pip*) a tiež povoliť *ipywidgets* doplnok v *Jupyter notebook*:
> ```
> jupyter nbextension enable --py widgetsnbextension
> ```

In [3]:
import math
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
%matplotlib inline

## Názorná ukážka, o čo dnes pôjde

-----------

- *Príklad*: výpočet plochy pod grafom funkcie $y=2\sin(5x) - \frac{x}{6}$ na intervale $\langle0,\,2\pi\rangle$

![Čo chceme vypočítať](img/y_fill.png)

In [4]:
x = np.linspace(0, 2 * np.pi, num=200)
y = 2 * np.sin(5 * x) - x / 6

def update(n=10):
    fig, ax = plt.subplots(figsize=(13, 6))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    x_stairs = np.linspace(0, 2 * np.pi, num=n)
    y_stairs = 2 * np.sin(5 * x_stairs) - x_stairs / 6
    
    x_step = x_stairs[1] - x_stairs[0]
    area = np.sum(y_stairs * x_step)
    
    ax.plot(x, y, lw=3, c='r')
    ax.fill_between(x_stairs, y_stairs, step='pre', alpha=.5)
    ax.plot(x_stairs, y_stairs, ds='steps', lw=3)
    
    
    plt.title('Počet krokov: {:d}{}Vypočítaná plocha: {:.5f}'.format(n - 1, 10 * ' ', area), fontdict={'fontsize': 25})

    fig.canvas.draw()

widgets.interact(update, n=(2, 200, 6));

interactive(children=(IntSlider(value=10, description='n', max=200, min=2, step=6), Output()), _dom_classes=('…

### Výpočet

- Presný: [Riemannov integrál](https://en.wikipedia.org/wiki/Riemann_integral)

In [5]:
# Using sympy
x = symbols('x')
integrate(2 * sin(5 * x) - x / 6, (x, 0, 2 * np.pi))

-3.28986813369645

- Odhad: Integrálny súčet na konečnom delení intervalu

#### Riešenie č. 1

- Pripomeňme si predpis funkcie: $y=2\sin(5x) - \frac{x}{6}$ a interval, na ktorom plochu počítame $\langle0,\,2\pi\rangle$

In [6]:
# An uniform partition of the interval
x1 = 0.
x2 = math.pi
x3 = 2 * math.pi

# Function values at the points of the partition
y1 = 2 * math.sin(5 * x1) - x1 / 6
y2 = 2 * math.sin(5 * x2) - x2 / 6
y3 = 2 * math.sin(5 * x3) - x3 / 6

# Sum of the rectangle areas
area = (x2 - x1) * y2 + (x3 - x2) * y3

print(area)

-4.9348022005446825


- Aj pre veľmi hrubé delenie intervalu je toto riešenie *veľmi zdĺhavé*
    - Prepisovanie veľmi podobných častí kódu niekoľkokrát za sebou
    
#### Riešenie č. 2

In [7]:
def linspace(start, stop, n):
    '''
    We do not have to calculate the points of a partition by hand!
    '''
    i = 1
    while i <= n:
        yield start + i * (stop - start) / n
        i += 1
        
xs = list(linspace(0, 2 * math.pi, int(2e5))) # One variable now serves for 200 000 values!
ys = [] # An empty variable of type 'list'

for x in xs: # We use a loop to perform the same operation many times
    ys.append(2 * math.sin(5 * x) - x / 6)
    
if len(xs) != len(ys): # Are the lists of the same length?
    print('Lists xs and ys contain different number of elements!')

area = 0.
for i in range(len(xs)):
    step = xs[1] - xs[0]
    area += ys[i] * step
    
print(area)

-3.289884583037101


- Použitie rôznych **typov** premenných na rôzne účely
- Použitie **cyklov** na opakovanie podobných operácií mnohokrát
- Použitie **funkcií** a **generátorov** na zapuzdrenie podprogramov
- **Parametrizácia** a **modularita** celého programu

## Vetvenie programu

-----------

- Viacero možností prechodu programom v závislosti na určitých *podmienkach*.

### Podmienka *if*

```python
if logický_výraz:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
```

#### Príklad: Výpis, či je v premennej _a_ uložené celé číslo

In [8]:
a = 'b'
if type(a) is int:
    print('Variable a represents an integer.')

### Podmienka *if* $-$ *else*

```python
if logický_výraz:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
else:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
```

#### Príklad: Porovnanie dvoch celých čísel

In [9]:
a = 5
b = 7

if a > b:
    print('a > b')
else:
    print('a <= b')

a <= b


### Podmienka *if* $-$ *elif* $-$ *else*

```python
if logický_výraz:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
elif logický_výraz:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
...
else:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
```

#### Príklad: Porovnanie dvoch celých čísel

In [10]:
a = 5
b = 7

if a > b:
    print('a > b')
elif a == b:
    print('a = b')
else:
    print('a < b')

a < b


> **Úloha 1**: (2 min)
> - Máme k dispozícii program, ktorý si od užívateľa vypýta celé čislo. Dopíšte do neho výpis reťazca `'Even'` v prípade, že ide o párne (sudé) číslo a výpis reťazca `'Odd'` v prípade, že ide o nepárne (liché) číslo.

In [11]:
n = int(input('Enter an integer: '))

##################
# Your code here #
##################

Enter an integer: 7


> **Riešenie:**

In [12]:
n = int(input('Enter an integer: '))

if n % 2 == 0:
    print('Even')
else:
    print('Odd')

Enter an integer: 7
Odd


## Cykly

-----------------

- Opakovanie podprogramu viackrát

### Cyklus *for*

- Pre vopred známy počet opakovaní

```python
for <premenná> in <iterovateľný_objekt>:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
```

#### Príklad: Výpis 7 trpaslíkov

In [13]:
for i in range(7):
    print('trpaslík ' + str(i + 1))

trpaslík 1
trpaslík 2
trpaslík 3
trpaslík 4
trpaslík 5
trpaslík 6
trpaslík 7


### Cyklus *while*

- Typicky pre vopred neznámy alebo nekonečný počet opakovaní

```python
while <logický_výraz_vyhodnocovaný_v_každom_cykle>:
    <príkaz>
    <príkaz>
    ...
    <príkaz>
```

- Možnosť vystúpiť z cyklu: príkaz `break`.

#### Príklad: Detekcia hrušky medzi jablkami

In [14]:
def fruit(n_pieces):
    for i in range(n_pieces):
        yield np.random.choice(['apple', 'pear'], p=[.9, .1])
        
fruitgen = fruit(10000)

f = ''
while True:
    f = next(fruitgen)
    if f == 'apple':
        print('apple')
    else:
        print('>>>>>>>>>>>>>>>>>pear<<<<<<<<<<<<<<<<<<<<')
        break


apple
apple
apple
apple
apple
>>>>>>>>>>>>>>>>>pear<<<<<<<<<<<<<<<<<<<<


## Príklady z [CW](https://cw.fel.cvut.cz/wiki/courses/bab37zpr/tutorials/lab02)

-----------------

1. Vypíšte čísla od 1 do 10 s využitím cyklu `for`

In [15]:
for i in range(10):
    print(i + 1)

1
2
3
4
5
6
7
8
9
10


2. Vypíšte čísla od 1 do 10 s využitím cyklu `while`

In [16]:
i = 1
while i <= 10:
    print(i)
    i += 1

1
2
3
4
5
6
7
8
9
10


3. Vypíšte všetky párne čísla od 1 do $n$, kde $n$ zadá užívateľ

In [17]:
n = int(input('Zadaj n: '))
even = []

for i in range(1, n + 1):
    if not i % 2:
        even.append(i)

print(even)

Zadaj n: 8
[2, 4, 6, 8]


4. Vypíšte súčin všetkých čísel z predchádzajúceho kroku

In [19]:
if len(even) < 1:
    print('There are no numbers in the list.')
else:
    product = 1
    for number in even:
        product *= number
    print(product)

384


5. Vypíšte rad $n$ čísel v tvare $1, -1, 1, -1, \ldots$

In [20]:
n = 100
series = ''
for i in range(n - 1):
    series += str((-1)**i) + ','
series += str((-1)**(n - 1))
    
print(series)

1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1


6. Napíšte program, ktorý umožní výpočet Ludolfovho čísla ako súčtu Leibnizovho radu $$\pi=4\sum^{\infty}_{k=0}\frac{(-1)^k}{2k+1}$$

In [130]:
infinity = int(1e2)
pi = 0

for k in range(infinity):
    pi += 4 * (-1)**k / (2 * k + 1)
    
print('pi: ' + str(pi))

pi: 3.1315929035585537


## Funkcie

------------

<img src="img/funkcie_nacrt/funkcie_nacrt.png" width="600"/>

- **Opakované použitie** časti kódu

```python
def <názov_funkcie>(<parametre_funkcie>):
    <príkaz>
    <príkaz>
    ...
    <príkaz>
    
    return <výstup>
```
#### Príklad: Funkcia na výpočet obsahu trojuholníka z dĺžky jeho strán

In [21]:
def heron(a, b, c):
    s = (a + b + c) / 2
    A = math.sqrt(s * (s-a) * (s-b) * (s-c))
    
    return A

In [22]:
print(heron(3,4,5))

6.0


### Default hodnoty parametrov

- Priradenie default hodnoty pri definícii funkcie
- Možnosť volať funkciu **bez explicitného zadania hodnôt parametrov**

In [23]:
def f(a=5):
    print(a**2)
    
a = f()
b = f(9)

type(a)

25
81


NoneType

- Vytvorenie „nadštandardnej funkcionality“

In [24]:
def hypotenuse(cathetus1, cathetus2, sqrt_fun=math.sqrt):
    return sqrt_fun(cathetus1**2 + cathetus2**2)

print(hypotenuse(3, 4))

print(hypotenuse(3, 4, np.sqrt))

5.0
5.0


### Malé terminologické okienko

- **Parameter** je *premenná*, s ktorou pracuje *definícia* funkcie
- **Argument** je *hodnota premennej*, ktorú inštancia funkcie dostáva pri jej volaní

### *Argumenty* vs. *keyword argumenty*

- Funkcie je možné volať aj s explicitným vyjadrením názvov ich parametrov

In [25]:
print(hypotenuse(cathetus1=3, cathetus2=4))

5.0


- Výhody:
    - Možnosť *meniť poradie* parametrov
    - Pri funkciách s veľkým množstvom parametrov *prehľadnosť*
    
### Funkcie s premenným množstvom argumentov

```python
def f(<povinný_parameter_1><povinný_parameter_2,...,<povinný_parameter_3>, *args):
    ...
    return ...
```

- Namiesto zoznamu `args` možno dosadiť akékoľvek množstvo *pozičných* argumentov
    - Ich hodnoty budú brané s ohľadom na **poradie**, nemožno ich do funkcie dodávať s menami

In [26]:
def f(a, b, *args):
    product = a * b
    
    for arg in args:
        product *= arg
        
    return product


print(f(5, 6, 7, 9))

1890


### Funkcie s povinne-keyword argumentami

```python
def f(<povinný_parameter_1>, <povinný_parameter_2, ..., <povinný_parameter_3>, *args, <povinný_kwarg_1>, <povinný_kwarg_2>, ...):
    ...
    return ...
```

- Nie je nutné využiť variabilné argumenty ([PEP-3102](https://www.python.org/dev/peps/pep-3102/)):
```python
def f(<povinný_parameter_1>, <povinný_parameter_2, ..., <povinný_parameter_3>, *, <povinný_kwarg_1>, <povinný_kwarg_2>, ...):
    ...
    return ...
```

In [27]:
def f(a, b, *, c=8):
    return a * b * c

    
print(f(5, 6, c=7))

210


### Anonymné funkcie

- Bežnejšie označované ako *lambda* funkcie
- Skutočne *nemajú meno* :-)

#### Príklad: Návrat päťnásobku argumentu
*Pozn.:* Aby sme mohli anonymnú funkciu zavolať, tentokrát jej meno *priradíme* (správnejšie, no menej logicky povedané, priradíme *funkciu do mena*) 

In [28]:
f = lambda x: 5 * x

print(f(7))
print(f('a'))

35
aaaaa


- Využitie:
    - Najčastejšie asi pri volaní funkcií, ktoré majú *funkcie ako parametre*
    - Jednoduché klonovanie funkcií podobného určenia

In [29]:
print(hypotenuse(3, 4, lambda x: x / 2))

12.5


In [31]:
def orders(table):
    return lambda drink: 'One {} for table {:d}'.format(drink, table)

o5 = orders(5)
o14 = orders(14)

print(o5('beer'))
print(o14('coffee'))
print(o14('beer'))

One beer for table 5
One coffee for table 14
One beer for table 14


## Jedna samostatná úloha

---

> **Úloha 2**: (5 min)
> - Máme zoznam hodnôt s jednotkami *pF* a *A*.
> - Hodnoty v pikofaradoch predstavujú *kondenzátory*, hodnoty v ampéroch *zdroje*.
> - Chceme spočítať, koľko je v zozname kondenzátorov a koľko zdrojov.
> - Príklad: Pre vstup 
```console
['486 A', '99 A', '50 pF', '346 pF', '131 pF', '358 A', '482 A', '24 pF', '51 A', '407 A']
```
> by sme sa chceli dopočítať k 6 zdrojom a 4 kondenzátorom

> - *Nápoveda 1*: Získanie jednotiek z reťazca tvaru `'24 pF'` možno urobiť pomocou `'24 pF'.split(' ')[-1]`
> - *Nápoveda 2*: Aplikácia operácie na každý element poľa: funkcia `map`

In [141]:
l = [1, 2, 3, 4, 5]

print(list(map(lambda k: k**2, l)))

[1, 4, 9, 16, 25]


In [142]:
values = ['{:2d} {}'.format(value, unit) for value, unit in zip(list(np.random.randint(500, size=10)), list(np.random.choice(['pF', 'A'], 10)))]

##################
# Your code here #
##################

> **Riešenie:**

In [32]:
values = ['{:2d} {}'.format(value, unit) for value, unit in zip(list(np.random.randint(500, size=10)), list(np.random.choice(['pF', 'A'], 10)))]

units = list(map(lambda k: k.split(' ')[-1], values))

c = s = 0
for u in units:
    if u == 'pF':
        c += 1
    elif u == 'A':
        s += 1
        
print('There are ' + str(s) + ' sources and ' + str(c) + ' capacitors in the list.')

There are 6 sources and 4 capacitors in the list.


## Dokončenie úloh z CW

---

7. Vytvorte *funkciu* pre výpočet Ludolfovho čísla pomocou $n$ prvkov Leibnizovho radu

In [33]:
def pi_leibniz(n=1000):
    pi = 0
    
    for k in range(n):
        pi += 4 * (-1)**k / (2 * k + 1)
    
    return pi


print('pi: ' + str(pi_leibniz(int(1e5))))

pi: 3.1415826535897198


8. Zistite, koľko členov radu je treba zahrnúť, aby sme dostali $\pi$ s presnosťou na $10^{-6}$

In [34]:
precision = 1e-2
pi_true = np.pi

n = 1
while np.abs(pi_true - pi_leibniz(n)) > precision:
    n += 1

print(n)

100


### Jednoduchý automat na mince

- Napíšte program, ktorý pre zadanú čiastku (v celých korunách) vypíše najmenší počet mincí (20, 10, 5, 2, 1), z ktorých sa dá daná čiastka zložiť

In [35]:
def automat(amount):
    n_coins = 0
    coins = [20, 10, 5, 2, 1]
    
    for coin in coins:
        n_coins += amount // coin
        amount -= (amount // coin) * coin
        
    return n_coins


print(automat(50))

3


- Rozšírte program tak, aby vypisoval použité mince

In [36]:
def automat(amount):
    coins = [20, 10, 5, 2, 1]
    used_coins = ''
    
    for coin in coins:
        used_coins += (amount // coin) * (str(coin) + ' ')
        amount -= (amount // coin) * coin
        
    return used_coins


print(automat(107))

20 20 20 20 20 5 2 


- Nechajte program vypísať vloženú čiastku aj v iných menách

In [37]:
def exchange(amount, currency='euro'):
    rates = {'euro': .037,
             'chilean_peso': 34.14,
             'bitcoin': 4e-6,
             'australian_dollar': .061,
             'belize_dollar': .088} # 1 CZK == ...
    
    if currency in rates.keys():
        return amount * rates[currency]
    else:
        print('No evidence.')
        return None
    

amount = 1000
currency = 'bitcoin'
print(str(amount) + ' CZK = ' + str(exchange(amount, currency)) + ' ' + currency)

1000 CZK = 0.004 bitcoin


## Čo ďalej

---

- Ešte raz si prejsť dnešné cvičenie, pochopenie dnešnej látky je naozaj dôležité.
- Naprogramovať si zopár vlastných úloh
    - Tip 1: Skúste si prejsť zopár úloh z [KSP](https://ksp.mff.cuni.cz/tasks/32/tasks1.html)
    - Tip 2: Výpočet $\pi$ metódou Monte Carlo
        - Trochu nad rámec dnešného cvičenia (je potrebné použiť nejaký generátor vzoriek)
    - Tip 3: Hľadanie prvočísel menších než $N$ pomocou algoritmu [Eratosthenovho sita](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes).
        - Jednoduchšia úloha